Color classification of images using CNN. Soumya and Anish, Jadavpur University, 2018.

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Download and unzip ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-02-17 05:29:15--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.4.95.48, 52.2.175.150, 52.201.75.180, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.4.95.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  3.40MB/s    in 1.5s    

2019-02-17 05:29:17 (3.40 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
# Run Tensorboard
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [ ]:
# Run ngrok
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
# Get URL
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://769038b2.ngrok.io


In [ ]:
# Run the CNN Model

from time import time

from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import AveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense,Activation
from keras.layers import LeakyReLU

from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard

from keras.preprocessing.image import ImageDataGenerator

# initalizing CNN
classifier=Sequential(name='ColorCNN')

# defining convolution layer one
classifier.add(Convolution2D(12,(7,7),
               input_shape=(48,48,3),
		           padding='same'))

# activation layer
classifier.add(Activation('relu'))

# pooling layer one
classifier.add(AveragePooling2D(pool_size=(3,3)))

# defining convolution layer two
#classifier.add(Convolution2D(6,(5,5),
 #              input_shape=(16,16,3),
	#	           padding='same'))

# activation layer
#classifier.add(Activation('sigmoid'))

# pooling layer two
#classifier.add(AveragePooling2D(pool_size=(3,3)))

# flattening
classifier.add(Flatten())

# full connection
classifier.add(Dense(activation='relu', units=256))
classifier.add(Dense(activation='softmax', units=6))

# compile
classifier.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

# image preprocessing and fitting to CNN
train_datagen=ImageDataGenerator(rescale=1./255)
'''
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True,
                                 featurewise_center=True)
'''


itr=train_datagen.flow_from_directory('/content/drive/My Drive/dataset',
                                       target_size=(48,48),
                                       color_mode='rgb',
                                       batch_size=18000,
                                       class_mode='categorical',
                                       shuffle=True)

X, y = itr.next()


# collecting data in tensorboard
tensorboard = TensorBoard(log_dir=LOG_DIR+'/{}'.format(time()),
                          histogram_freq=1,
			                    write_graph=True,
			  			  			  	write_grads=True,
			  			  			    batch_size=60,
			  			  			    write_images=True)

Found 18000 images belonging to 6 classes.


In [ ]:
# running the model
classifier.fit(X, y,
               batch_size=60,
               epochs=25,
               verbose=1,
               callbacks=[tensorboard],
               validation_split=float(1)/3)

Instructions for updating:
Use tf.cast instead.
Train on 12000 samples, validate on 6000 samples
Epoch 1/25
12000/12000 [==============================] - 5s 436us/step - loss: 0.5662 - acc: 0.7952 - val_loss: 0.4607 - val_acc: 0.8402
Epoch 2/25
12000/12000 [==============================] - 2s 191us/step - loss: 0.4404 - acc: 0.8460 - val_loss: 0.4143 - val_acc: 0.8568
Epoch 3/25
12000/12000 [==============================] - 2s 184us/step - loss: 0.3901 - acc: 0.8672 - val_loss: 0.4342 - val_acc: 0.8533
Epoch 4/25
12000/12000 [==============================] - 2s 189us/step - loss: 0.3472 - acc: 0.8802 - val_loss: 0.4352 - val_acc: 0.8482
Epoch 5/25
12000/12000 [==============================] - 2s 190us/step - loss: 0.3126 - acc: 0.8917 - val_loss: 0.3968 - val_acc: 0.8687
Epoch 6/25
12000/12000 [==============================] - 2s 190us/step - loss: 0.2919 - acc: 0.9001 - val_loss: 0.4216 - val_acc: 0.8650
Epoch 7/25
12000/12000 [==============================] - 2s 189us/step - l